# Spike and Burst Analysis with Visibility Graphs

This notebook demonstrates detection of spikes and bursts in an ABF file, classifies bursts into different types, and constructs visibility graphs with 2D and 3D embeddings for bursts. It also saves all the burst information and graph data for further analysis.

In [ ]:
# Load libraries
import pyabf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx  # Creates and analyses graphs
from scipy.signal import find_peaks
from sklearn.decomposition import TruncatedSVD  # Dimensionality reduction
%matplotlib widget

## Step 1: Load ABF file and concatenate sweeps
ABF (Axon Binary File) is a format used in electrophysiology for patch-clamp or intracellular recordings.
We combine all sweeps into a single continuous voltage trace for analysis.

**Expected output:** Prints file name, number of sweeps, and total recording duration.

In [ ]:
file_path = "bursting/cell89basal.abf"
abf = pyabf.ABF(file_path)

# Concatenate sweeps
signal = np.concatenate([abf.setSweep(i) or abf.sweepY for i in range(abf.sweepCount)])
dt = 1.0 / abf.dataRate
time = np.arange(len(signal)) * dt

print(f"File: {file_path} | sweeps: {abf.sweepCount} | total length: {time[-1]:.2f} s")

## Step 2: Spike detection
Detect spikes using a voltage threshold (-35 mV).

In [ ]:
threshold = -35  # mV
spike_indices, _ = find_peaks(signal, height=threshold)
spike_times = time[spike_indices]

## Step 3: Burst detection
Bursts are sequences of spikes where the interspike interval (ISI) is below 0.3 s.

In [ ]:
isi = np.diff(spike_times)
burst_threshold = 0.3  # s
bursts = []
current_burst = [0]

for i in range(1, len(spike_times)):
    if isi[i-1] < burst_threshold:
        current_burst.append(i)
    else:
        if len(current_burst) > 1:
            bursts.append(current_burst)
        current_burst = [i]
if len(current_burst) > 1:
    bursts.append(current_burst)

print(f"Detected {len(bursts)} bursts")

## Step 4: Calculate internal ISI for each burst (ms)

In [ ]:
isi_per_spike_burst = np.zeros(len(spike_times))
for burst in bursts:
    for i, idx in enumerate(burst):
        isi_per_spike_burst[idx] = 0 if i==0 else (spike_times[idx]-spike_times[burst[i-1]])*1000

## Step 5: Burst classification
Classify bursts as Square Wave, Parabolic, or Other using voltage baseline comparison.

In [ ]:
square_wave_bursts, parabolic_bursts, other_bursts = [], [], []
burst_types = {}

for i, burst in enumerate(bursts):
    burst_mask = (time >= spike_times[burst[0]]) & (time <= spike_times[burst[-1]])
    burst_min = np.min(signal[burst_mask])
    prev_mean = np.mean(signal[(time > spike_times[bursts[i-1][-1]]) & (time < spike_times[burst[0]])]) if i>0 else np.nan
    next_mean = np.mean(signal[(time > spike_times[burst[-1]]) & (time < spike_times[bursts[i+1][0]])]) if i<len(bursts)-1 else np.nan
    inter_mean = np.nanmean([prev_mean, next_mean])
    if burst_min > inter_mean:
        square_wave_bursts.append(burst)
        burst_types[tuple(burst)] = "Square Wave"
    elif burst_min < inter_mean:
        parabolic_bursts.append(burst)
        burst_types[tuple(burst)] = "Parabolic"
    else:
        other_bursts.append(burst)
        burst_types[tuple(burst)] = "Other"

## Step 6: Save burst info to CSV

In [ ]:
burst_list = []
for idx, burst in enumerate(bursts):
    burst_type = burst_types[tuple(burst)]
    burst_list.append([idx+1, spike_times[burst[0]], spike_times[burst[-1]], burst_type])

df_bursts_all = pd.DataFrame(burst_list, columns=["Burst_Number","Start_Time_s","End_Time_s","Type"])
df_bursts_all.to_csv("burst_basic_info_cell89_all_bursts.csv", index=False)
print(df_bursts_all.head(10))

## Step 7: Visibility Graph and Embeddings

In [ ]:
colors_map = {"Square Wave":"blue", "Parabolic":"green", "Other":"orange"}
nodes_list, edges_list = [], []

for b_idx, burst in enumerate(bursts):
    burst_type = burst_types[tuple(burst)]
    x_peaks = np.arange(len(burst))
    y_peaks = isi_per_spike_burst[burst]

    # Visibility graph
    G = nx.Graph()
    G.add_nodes_from(range(len(burst)))
    for a in range(len(x_peaks)):
        for b in range(a+1, len(x_peaks)):
            visible = True
            for c in range(a+1,b):
                y_line = y_peaks[b]+(y_peaks[a]-y_peaks[b])*(x_peaks[b]-x_peaks[c])/(x_peaks[b]-x_peaks[a])
                if y_peaks[c] >= y_line:
                    visible=False
                    break
            if visible:
                G.add_edge(a,b)
                edges_list.append([b_idx+1, burst_type, a, b])

    # Embeddings
    A = nx.to_numpy_array(G)
    n_dim = min(3,A.shape[0])
    embedding_2d = TruncatedSVD(n_components=2, random_state=42).fit_transform(A)
    embedding_3d = TruncatedSVD(n_components=n_dim, random_state=42).fit_transform(A) if n_dim>=3 else np.zeros((len(burst),3))

    for i in range(len(burst)):
        nodes_list.append([b_idx+1, burst_type, i, embedding_2d[i,0], embedding_2d[i,1], embedding_3d[i,0], embedding_3d[i,1], embedding_3d[i,2], burst[i]])

    # Plot first 14 bursts
    if b_idx < 14:
        fig = plt.figure(figsize=(12,10))
        ax0 = fig.add_subplot(3,1,1)
        for u,v in G.edges():
            ax0.plot([x_peaks[u], x_peaks[v]], [y_peaks[u], y_peaks[v]], 'gray', alpha=0.5)
        ax0.scatter(x_peaks, y_peaks, color=colors_map[burst_type], s=40)
        ax0.set_title(f"Burst {b_idx+1} Visibility graph ({burst_type})")
        ax0.set_xlabel("Spike index")
        ax0.set_ylabel("ISI (ms)")
        ax0.grid(True)

        ax1 = fig.add_subplot(3,1,2)
        for i in range(len(burst)):
            ax1.scatter(embedding_2d[i,0], embedding_2d[i,1], color=colors_map[burst_type], s=50)
            ax1.text(embedding_2d[i,0]+0.01, embedding_2d[i,1]+0.01, str(i), fontsize=8)
        for u,v in G.edges():
            ax1.plot([embedding_2d[u,0], embedding_2d[v,0]], [embedding_2d[u,1], embedding_2d[v,1]], 'r-', alpha=0.3)
        ax1.set_title(f"Burst {b_idx+1} Embedding 2D")
        ax1.grid(True)

        if embedding_3d is not None:
            ax2 = fig.add_subplot(3,1,3, projection='3d')
            for i in range(len(burst)):
                ax2.scatter(embedding_3d[i,0], embedding_3d[i,1], embedding_3d[i,2], color=colors_map[burst_type], s=40)
                ax2.text(embedding_3d[i,0], embedding_3d[i,1], embedding_3d[i,2], str(i), fontsize=8)
            for u,v in G.edges():
                ax2.plot([embedding_3d[u,0], embedding_3d[v,0]], [embedding_3d[u,1], embedding_3d[v,1]], [embedding_3d[u,2], embedding_3d[v,2]], 'r-', alpha=0.3)
            ax2.set_title(f"Burst {b_idx+1} Embedding 3D")

        plt.tight_layout()
        plt.show()

# Save CSVs
df_nodes = pd.DataFrame(nodes_list, columns=["Burst_Number","Type","Node_ID","X_2D","Y_2D","X_3D","Y_3D","Z_3D","Spike_Global_Index"])
df_edges = pd.DataFrame(edges_list, columns=["Burst_Number","Type","Node1_ID","Node2_ID"])
df_nodes.to_csv("burst_nodes_all.csv", index=False)
df_edges.to_csv("burst_edges_all.csv", index=False)